# Capítulo 2: Anatomia de um DataChunk

## Introdução

O **DataChunk** é a estrutura de dados fundamental do DuckDB. É o "bloco de construção" que permite a execução vetorizada eficiente.

Neste capítulo, vamos explorar:
1. O que é um DataChunk e como ele é estruturado
2. Tipos de vetores: Flat, Constant, Dictionary e Sequence
3. Como o DuckDB otimiza memória com representações especiais
4. Implementação prática de DataChunks em Python

In [ ]:
# Instalação das dependências
!pip install duckdb pandas numpy matplotlib seaborn -q

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass
from typing import List, Any, Optional
from enum import Enum
import sys

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## 2.1 O que é um DataChunk?

Um **DataChunk** é uma coleção de **vetores** (colunas) que compartilham o mesmo número de linhas.

```
DataChunk (1024 linhas)
┌─────────────┬─────────────┬─────────────┐
│  Vector 0   │  Vector 1   │  Vector 2   │
│  (int64)    │  (float64)  │  (string)   │
├─────────────┼─────────────┼─────────────┤
│     1       │    1.5      │   "Alice"   │
│     2       │    2.5      │   "Bob"     │
│     3       │    3.5      │   "Carol"   │
│    ...      │    ...      │    ...      │
│   1024      │  1024.5     │   "Zara"    │
└─────────────┴─────────────┴─────────────┘
```

### Características principais:
- **Tamanho fixo**: Tipicamente 1024 ou 2048 linhas
- **Layout colunar**: Cada vetor armazena dados de uma coluna
- **Cache-friendly**: Tamanho otimizado para caber no cache L1/L2

In [ ]:
# Simulação de um DataChunk em Python

class VectorType(Enum):
    FLAT = "FLAT"           # Vetor normal com todos os valores
    CONSTANT = "CONSTANT"   # Todos os valores são iguais
    DICTIONARY = "DICTIONARY"  # Valores codificados com dicionário
    SEQUENCE = "SEQUENCE"   # Sequência aritmética (start, increment)


@dataclass
class Vector:
    """Representação de um vetor do DuckDB"""
    data: np.ndarray
    vector_type: VectorType
    validity_mask: Optional[np.ndarray] = None  # Para valores NULL
    
    # Metadados para tipos especiais
    constant_value: Any = None
    dictionary: Optional[np.ndarray] = None
    sequence_start: int = 0
    sequence_increment: int = 1
    
    def __len__(self):
        if self.vector_type == VectorType.CONSTANT:
            return len(self.data) if self.data is not None else 0
        return len(self.data)
    
    def get_value(self, idx: int) -> Any:
        """Obtém valor no índice especificado"""
        if self.validity_mask is not None and not self.validity_mask[idx]:
            return None
        
        if self.vector_type == VectorType.CONSTANT:
            return self.constant_value
        elif self.vector_type == VectorType.DICTIONARY:
            return self.dictionary[self.data[idx]]
        elif self.vector_type == VectorType.SEQUENCE:
            return self.sequence_start + idx * self.sequence_increment
        else:  # FLAT
            return self.data[idx]
    
    def memory_usage(self) -> int:
        """Calcula uso de memória em bytes"""
        if self.vector_type == VectorType.CONSTANT:
            return 8  # Apenas o valor constante
        elif self.vector_type == VectorType.SEQUENCE:
            return 16  # start + increment
        elif self.vector_type == VectorType.DICTIONARY:
            return self.data.nbytes + self.dictionary.nbytes
        else:
            return self.data.nbytes


class DataChunk:
    """Representação de um DataChunk do DuckDB"""
    STANDARD_VECTOR_SIZE = 2048
    
    def __init__(self, size: int = STANDARD_VECTOR_SIZE):
        self.size = size
        self.vectors: List[Vector] = []
    
    def add_vector(self, vector: Vector):
        self.vectors.append(vector)
    
    def get_column(self, idx: int) -> Vector:
        return self.vectors[idx]
    
    def total_memory(self) -> int:
        return sum(v.memory_usage() for v in self.vectors)

print("Classes DataChunk e Vector definidas!")

## 2.2 Flat Vector - O Tipo Básico

O **Flat Vector** é a representação mais simples: um array contíguo de valores.

```
Flat Vector (int64)
┌───┬───┬───┬───┬───┬───┬───┬───┐
│ 1 │ 2 │ 3 │ 4 │ 5 │ 6 │ 7 │ 8 │  ... (até 2048)
└───┴───┴───┴───┴───┴───┴───┴───┘
  8 bytes cada = 16KB para 2048 int64
```

In [ ]:
# Criando um Flat Vector
def create_flat_vector(data: np.ndarray) -> Vector:
    return Vector(
        data=data,
        vector_type=VectorType.FLAT
    )

# Exemplo
size = 2048
flat_data = np.arange(1, size + 1, dtype=np.int64)
flat_vector = create_flat_vector(flat_data)

print(f"Flat Vector criado com {len(flat_vector)} elementos")
print(f"Primeiros 10 valores: {[flat_vector.get_value(i) for i in range(10)]}")
print(f"Uso de memória: {flat_vector.memory_usage():,} bytes ({flat_vector.memory_usage()/1024:.1f} KB)")

## 2.3 Constant Vector - Compressão de Valores Repetidos

Quando **todos os valores são iguais**, o DuckDB usa um **Constant Vector** que armazena apenas um único valor!

```
Constant Vector (valor = "ACTIVE")
┌────────────────────────────────┐
│  constant_value = "ACTIVE"     │  <- Apenas 1 valor armazenado!
└────────────────────────────────┘

Lógica: get_value(i) sempre retorna "ACTIVE", independente de i
```

**Economia**: Em vez de 2048 strings "ACTIVE", armazena apenas 1!

In [ ]:
# Criando um Constant Vector
def create_constant_vector(value: Any, size: int) -> Vector:
    return Vector(
        data=np.zeros(size),  # Placeholder para compatibilidade
        vector_type=VectorType.CONSTANT,
        constant_value=value
    )

# Exemplo: Coluna status com todos os valores "ACTIVE"
constant_vector = create_constant_vector("ACTIVE", 2048)

print(f"Constant Vector criado para {len(constant_vector)} elementos")
print(f"Valor em índice 0: {constant_vector.get_value(0)}")
print(f"Valor em índice 500: {constant_vector.get_value(500)}")
print(f"Valor em índice 2047: {constant_vector.get_value(2047)}")
print(f"\nUso de memória: {constant_vector.memory_usage()} bytes")

# Comparando com Flat Vector equivalente
flat_strings = np.array(["ACTIVE"] * 2048)
print(f"\nSe fosse Flat Vector de strings: ~{sys.getsizeof('ACTIVE') * 2048:,} bytes")
print(f"Economia: ~{(sys.getsizeof('ACTIVE') * 2048) / constant_vector.memory_usage():.0f}x menos memória!")

## 2.4 Dictionary Vector - Compressão por Dicionário

Quando há **poucos valores distintos** (baixa cardinalidade), o DuckDB usa **Dictionary Encoding**.

```
Dictionary Vector
┌─────────────────────┐      ┌─────────────────┐
│  Dicionário         │      │  Índices        │
├─────────────────────┤      ├─────────────────┤
│  0 -> "São Paulo"   │      │  0              │
│  1 -> "Rio de Janeiro"│    │  1              │
│  2 -> "Curitiba"    │      │  0              │
│  3 -> "Salvador"    │      │  2              │
└─────────────────────┘      │  1              │
     4 strings               │  3              │
                             │  0              │
                             │  ...            │
                             └─────────────────┘
                             2048 inteiros pequenos
```

In [ ]:
# Criando um Dictionary Vector
def create_dictionary_vector(values: np.ndarray) -> Vector:
    """Converte array de valores em Dictionary Vector"""
    unique_values, indices = np.unique(values, return_inverse=True)
    
    # Usa tipo menor para índices se possível
    if len(unique_values) < 256:
        indices = indices.astype(np.uint8)
    elif len(unique_values) < 65536:
        indices = indices.astype(np.uint16)
    else:
        indices = indices.astype(np.uint32)
    
    return Vector(
        data=indices,
        vector_type=VectorType.DICTIONARY,
        dictionary=unique_values
    )

# Exemplo: Coluna de cidades (baixa cardinalidade)
cidades = ["São Paulo", "Rio de Janeiro", "Curitiba", "Salvador", "Belo Horizonte"]
dados_cidades = np.random.choice(cidades, size=2048)

dict_vector = create_dictionary_vector(dados_cidades)

print(f"Dictionary Vector criado com {len(dict_vector)} elementos")
print(f"Valores únicos no dicionário: {len(dict_vector.dictionary)}")
print(f"Dicionário: {dict_vector.dictionary}")
print(f"\nPrimeiros 10 índices: {dict_vector.data[:10]}")
print(f"Primeiros 10 valores decodificados: {[dict_vector.get_value(i) for i in range(10)]}")

# Comparação de memória
flat_memory = sum(len(s.encode('utf-8')) for s in dados_cidades)
dict_memory = dict_vector.memory_usage()

print(f"\nMemória Flat Vector: ~{flat_memory:,} bytes")
print(f"Memória Dictionary Vector: {dict_memory:,} bytes")
print(f"Economia: {flat_memory/dict_memory:.1f}x menos memória!")

## 2.5 Sequence Vector - Sequências Aritméticas

Para sequências como `1, 2, 3, 4, ...` o DuckDB armazena apenas o **início** e o **incremento**.

```
Sequence Vector
┌─────────────────────────┐
│  start = 1              │
│  increment = 1          │
└─────────────────────────┘

get_value(i) = start + (i * increment)
get_value(0) = 1 + (0 * 1) = 1
get_value(5) = 1 + (5 * 1) = 6
```

**Uso comum**: Colunas de ID geradas automaticamente, `generate_series()`

In [ ]:
# Criando um Sequence Vector
def create_sequence_vector(start: int, increment: int, size: int) -> Vector:
    return Vector(
        data=np.zeros(size),  # Placeholder
        vector_type=VectorType.SEQUENCE,
        sequence_start=start,
        sequence_increment=increment
    )

# Exemplo: Coluna ID de 1 a 2048
seq_vector = create_sequence_vector(start=1, increment=1, size=2048)

print(f"Sequence Vector: start={seq_vector.sequence_start}, increment={seq_vector.sequence_increment}")
print(f"\nValores calculados:")
for i in [0, 1, 10, 100, 2047]:
    print(f"  get_value({i}) = {seq_vector.get_value(i)}")

print(f"\nUso de memória: {seq_vector.memory_usage()} bytes")
print(f"Flat Vector equivalente: {2048 * 8:,} bytes")
print(f"Economia: {(2048 * 8) / seq_vector.memory_usage():,.0f}x menos memória!")

## 2.6 Visualização: Comparação de Uso de Memória

In [ ]:
# Comparação visual de uso de memória
size = 2048

# Cenário 1: Coluna numérica sequencial (ID)
flat_seq_memory = size * 8  # int64
seq_memory = 16  # start + increment

# Cenário 2: Coluna de strings constantes
flat_const_memory = size * 50  # ~50 bytes por string média
const_memory = 50  # Uma string

# Cenário 3: Coluna de baixa cardinalidade (5 valores distintos)
flat_dict_memory = size * 50  # strings completas
dict_memory = 5 * 50 + size * 1  # dicionário + índices uint8

scenarios = ['Sequencial (ID)', 'Constante (Status)', 'Baixa Cardinalidade']
flat_memories = [flat_seq_memory, flat_const_memory, flat_dict_memory]
optimized_memories = [seq_memory, const_memory, dict_memory]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
x = np.arange(len(scenarios))
width = 0.35

ax1 = axes[0]
bars1 = ax1.bar(x - width/2, [m/1024 for m in flat_memories], width, label='Flat Vector', color='#e74c3c')
bars2 = ax1.bar(x + width/2, [m/1024 for m in optimized_memories], width, label='Otimizado', color='#27ae60')

ax1.set_ylabel('Memória (KB)')
ax1.set_title('Uso de Memória: Flat vs Otimizado')
ax1.set_xticks(x)
ax1.set_xticklabels(scenarios)
ax1.legend()
ax1.set_yscale('log')

# Gráfico de economia
ax2 = axes[1]
savings = [f/o for f, o in zip(flat_memories, optimized_memories)]
colors = ['#3498db', '#9b59b6', '#1abc9c']
bars = ax2.bar(scenarios, savings, color=colors)
ax2.set_ylabel('Fator de Economia (x vezes)')
ax2.set_title('Economia de Memória com Vetores Especiais')

# Adicionar valores nas barras
for bar, saving in zip(bars, savings):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
             f'{saving:.0f}x', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 2.7 Validity Mask - Tratamento de NULLs

O DuckDB usa uma **máscara de bits** para indicar valores NULL, sem desperdiçar espaço.

```
Vetor de dados:     [10, 20, 30, 40, 50, 60, 70, 80]
Validity Mask:      [ 1,  1,  0,  1,  1,  0,  1,  1]
                           ^           ^
                         NULL        NULL

Valores reais:      [10, 20, NULL, 40, 50, NULL, 70, 80]
```

In [ ]:
# Implementando Validity Mask
def create_vector_with_nulls(data: np.ndarray, null_indices: List[int]) -> Vector:
    """Cria vetor com máscara de validade para NULLs"""
    validity = np.ones(len(data), dtype=bool)
    validity[null_indices] = False
    
    return Vector(
        data=data,
        vector_type=VectorType.FLAT,
        validity_mask=validity
    )

# Exemplo
data = np.array([10, 20, 30, 40, 50, 60, 70, 80], dtype=np.int64)
null_positions = [2, 5]  # Índices 2 e 5 são NULL

vector_with_nulls = create_vector_with_nulls(data, null_positions)

print("Dados armazenados:", vector_with_nulls.data)
print("Máscara de validade:", vector_with_nulls.validity_mask)
print("\nValores interpretados:")
for i in range(len(data)):
    value = vector_with_nulls.get_value(i)
    print(f"  Índice {i}: {value if value is not None else 'NULL'}")

## 2.8 DataChunk Completo - Juntando Tudo

In [ ]:
# Criando um DataChunk completo simulando uma tabela
def create_sample_datachunk(size: int = 2048) -> DataChunk:
    """Cria um DataChunk de exemplo com diferentes tipos de vetores"""
    chunk = DataChunk(size)
    
    # Coluna 1: ID (Sequence Vector)
    chunk.add_vector(create_sequence_vector(1, 1, size))
    
    # Coluna 2: Status (Constant Vector) - todos "ACTIVE"
    chunk.add_vector(create_constant_vector("ACTIVE", size))
    
    # Coluna 3: Cidade (Dictionary Vector)
    cidades = np.random.choice(
        ["São Paulo", "Rio de Janeiro", "Curitiba", "Salvador"], 
        size=size
    )
    chunk.add_vector(create_dictionary_vector(cidades))
    
    # Coluna 4: Valor (Flat Vector)
    valores = np.random.uniform(100, 10000, size).astype(np.float64)
    chunk.add_vector(create_flat_vector(valores))
    
    return chunk

# Criar e analisar DataChunk
chunk = create_sample_datachunk(2048)

print("=" * 60)
print("ANÁLISE DO DATACHUNK")
print("=" * 60)

colunas = ["id", "status", "cidade", "valor"]

for i, (nome, vector) in enumerate(zip(colunas, chunk.vectors)):
    print(f"\nColuna '{nome}':")
    print(f"  Tipo: {vector.vector_type.value}")
    print(f"  Memória: {vector.memory_usage():,} bytes")
    print(f"  Exemplo: {vector.get_value(0)}, {vector.get_value(100)}, {vector.get_value(2047)}")

print(f"\n{'='*60}")
print(f"MEMÓRIA TOTAL DO CHUNK: {chunk.total_memory():,} bytes ({chunk.total_memory()/1024:.1f} KB)")
print(f"{'='*60}")

## 2.9 DuckDB em Ação - Observando DataChunks Reais

In [ ]:
# Conectar ao DuckDB e criar dados de teste
con = duckdb.connect(':memory:')

# Criar tabela com diferentes padrões de dados
con.execute("""
    CREATE TABLE vendas AS
    SELECT 
        i AS id,
        'ATIVO' AS status,  -- Constante
        CASE (i % 5)
            WHEN 0 THEN 'São Paulo'
            WHEN 1 THEN 'Rio de Janeiro'
            WHEN 2 THEN 'Curitiba'
            WHEN 3 THEN 'Salvador'
            ELSE 'Belo Horizonte'
        END AS cidade,  -- Baixa cardinalidade
        random() * 10000 AS valor,  -- Valores variados
        CASE WHEN random() > 0.9 THEN NULL ELSE random() * 100 END AS desconto  -- Com NULLs
    FROM generate_series(1, 100000) AS t(i)
""")

print("Tabela 'vendas' criada com 100.000 linhas")
con.execute("SELECT * FROM vendas LIMIT 5").df()

In [ ]:
# Analisar estatísticas e compressão
print("=== ESTATÍSTICAS DA TABELA ===")
stats = con.execute("""
    SELECT 
        COUNT(*) as total_linhas,
        COUNT(DISTINCT status) as status_distintos,
        COUNT(DISTINCT cidade) as cidades_distintas,
        COUNT(*) - COUNT(desconto) as descontos_null
    FROM vendas
""").df()
print(stats.to_string(index=False))

In [ ]:
# Ver plano de execução para entender como DuckDB processa
print("\n=== PLANO DE EXECUÇÃO ===")
plan = con.execute("""
    EXPLAIN ANALYZE 
    SELECT cidade, SUM(valor) 
    FROM vendas 
    WHERE status = 'ATIVO'
    GROUP BY cidade
""").df()
print(plan.to_string())

## 2.10 Exercícios Práticos

### Exercício 1: Identificar tipo de vetor ideal

In [ ]:
def suggest_vector_type(data: np.ndarray) -> str:
    """
    Analisa os dados e sugere o tipo de vetor mais eficiente.
    
    Implemente a lógica para:
    - CONSTANT: se todos valores são iguais
    - SEQUENCE: se é uma progressão aritmética
    - DICTIONARY: se cardinalidade < 10% do tamanho
    - FLAT: caso padrão
    """
    n = len(data)
    unique_vals = np.unique(data)
    n_unique = len(unique_vals)
    
    # Verificar se é constante
    if n_unique == 1:
        return "CONSTANT"
    
    # Verificar se é sequência aritmética
    if np.issubdtype(data.dtype, np.number):
        diffs = np.diff(data)
        if len(np.unique(diffs)) == 1:
            return f"SEQUENCE (start={data[0]}, inc={diffs[0]})"
    
    # Verificar cardinalidade
    if n_unique / n < 0.1:
        return f"DICTIONARY ({n_unique} valores únicos)"
    
    return "FLAT"

# Testes
test_cases = [
    ("IDs sequenciais", np.arange(1, 1001)),
    ("Todos 'ACTIVE'", np.array(["ACTIVE"] * 1000)),
    ("5 categorias", np.random.choice(["A", "B", "C", "D", "E"], 1000)),
    ("Valores aleatórios", np.random.randn(1000)),
    ("Sequência com step 2", np.arange(0, 2000, 2)),
]

print("ANÁLISE DE TIPOS DE VETOR IDEAIS")
print("=" * 50)
for name, data in test_cases:
    suggestion = suggest_vector_type(data)
    print(f"{name:25} -> {suggestion}")

### Exercício 2: Calcular economia de memória

In [ ]:
def calculate_memory_savings(data: np.ndarray) -> dict:
    """
    Calcula a economia de memória usando vetores especiais.
    Retorna dicionário com memórias e economia percentual.
    """
    n = len(data)
    flat_memory = data.nbytes
    
    # Criar vetores e comparar
    unique_vals = np.unique(data)
    
    # Memória Dictionary
    if len(unique_vals) < 256:
        index_size = 1
    elif len(unique_vals) < 65536:
        index_size = 2
    else:
        index_size = 4
    dict_memory = unique_vals.nbytes + n * index_size
    
    # Memória Constant (se aplicável)
    if len(unique_vals) == 1:
        const_memory = unique_vals.nbytes
    else:
        const_memory = flat_memory
    
    best_memory = min(flat_memory, dict_memory, const_memory)
    savings = (flat_memory - best_memory) / flat_memory * 100
    
    return {
        'flat_memory': flat_memory,
        'dict_memory': dict_memory,
        'const_memory': const_memory,
        'best_memory': best_memory,
        'savings_percent': savings
    }

# Análise
cidades = np.random.choice(["SP", "RJ", "MG", "BA", "PR"], 10000)
result = calculate_memory_savings(cidades)

print("ANÁLISE DE MEMÓRIA")
print(f"Flat Vector:       {result['flat_memory']:>10,} bytes")
print(f"Dictionary Vector: {result['dict_memory']:>10,} bytes")
print(f"Economia:          {result['savings_percent']:>10.1f}%")

## 2.11 Resumo do Capítulo

### Tipos de Vetores no DuckDB

| Tipo | Uso | Memória | Cenário |
|------|-----|---------|--------|
| **Flat** | Geral | O(n) | Dados variados |
| **Constant** | Valor único | O(1) | Colunas com mesmo valor |
| **Dictionary** | Baixa cardinalidade | O(k + n) | Categorias, status |
| **Sequence** | Progressões | O(1) | IDs, timestamps regulares |

### Principais Takeaways:

1. **DataChunk** é a unidade fundamental de processamento vetorizado
2. O DuckDB escolhe automaticamente o tipo de vetor mais eficiente
3. **Constant Vectors** são extremamente eficientes para valores repetidos
4. **Dictionary Encoding** reduz drasticamente memória para baixa cardinalidade
5. **Validity Masks** permitem tratamento eficiente de NULLs

In [ ]:
# Limpar recursos
con.close()